## Deploy Text Generation Model (Cohere Generate - Command Medium)

##### Prerequisites 

In [2]:
%%capture

!pip install cohere-sagemaker

#### Imports 

In [3]:
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import sagemaker
import logging
import boto3
import time

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


##### Setup logging 

In [4]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies 

In [5]:
logger.info(f'Using sagemaker=={sagemaker.__version__}')
logger.info(f'Using boto3=={boto3.__version__}')

Using sagemaker==2.173.0
Using boto3==1.28.14


#### Setup essentials

Mapping for Model Packages (initially only us-east-1 and eu-west-1 is supported)

In [8]:
model_package_map = {
    #'us-east-1': 'arn:aws:sagemaker:us-east-1:865070037744:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421',
    'eu-west-1': 'arn:aws:sagemaker:eu-west-1:985815980388:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421'
}

In [9]:
region = boto3.Session().region_name
logger.info(f'Region = {region}')

Region = eu-west-1


In [10]:
if region not in model_package_map.keys():
    raise Exception(f'Unsupported region = {region}')

In [12]:
MODEL_PACKAGE_ARN = model_package_map[region]
logger.info(f'Model package ARN = {MODEL_PACKAGE_ARN}')

Model package ARN = arn:aws:sagemaker:eu-west-1:985815980388:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421


In [13]:
ROLE = get_execution_role()
session = sagemaker.Session()
logger.info(f'Role = {ROLE}')

Role = arn:aws:iam::447598836120:role/service-role/AmazonSageMaker-ExecutionRole-20230206T112264


In [14]:
timestamp = int(time.time())
MODEL_NAME = f'cohere-medium-{timestamp}'

#### Create a SageMaker model

In [15]:
model = ModelPackage(role=ROLE, 
                     model_package_arn=MODEL_PACKAGE_ARN, 
                     sagemaker_session=session, 
                     name=MODEL_NAME)
model.__dict__

{'model_data': None,
 'image_uri': None,
 'predictor_cls': None,
 'name': 'cohere-medium-1691076016',
 '_base_name': None,
 'sagemaker_session': <sagemaker.session.Session at 0x7f45a902beb0>,
 '_sagemaker_config': None,
 'role': 'arn:aws:iam::447598836120:role/service-role/AmazonSageMaker-ExecutionRole-20230206T112264',
 'vpc_config': None,
 'endpoint_name': None,
 '_is_compiled_model': False,
 '_compilation_job_name': None,
 '_is_edge_packaged_model': False,
 'inference_recommender_job_results': None,
 'inference_recommendations': None,
 '_enable_network_isolation': False,
 'env': {},
 'model_kms_key': None,
 'image_config': None,
 'entry_point': None,
 'source_dir': None,
 'dependencies': [],
 'git_config': None,
 'container_log_level': 20,
 'bucket': None,
 'key_prefix': None,
 'uploaded_code': None,
 'repacked_model_data': None,
 'algorithm_arn': None,
 'model_package_arn': 'arn:aws:sagemaker:eu-west-1:985815980388:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c4

#### Deploy the text generation model as a SageMaker endpoint for real-time synchronous inference

In [16]:
NUM_INSTANCES = 1
INSTANCE_TYPE = 'ml.g5.xlarge'

In [17]:
%%time

model.deploy(NUM_INSTANCES, 
             INSTANCE_TYPE, 
             endpoint_name=MODEL_NAME)

Creating model with name: cohere-medium-1691076016
CreateModel request: {
    "ModelName": "cohere-medium-1691076016",
    "ExecutionRoleArn": "arn:aws:iam::447598836120:role/service-role/AmazonSageMaker-ExecutionRole-20230206T112264",
    "Containers": [
        {
            "ModelPackageName": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421"
        }
    ],
    "EnableNetworkIsolation": true
}


ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Caller is not subscribed to the marketplace offering.